# Materials Project search for further details

Materials Project entries were initially filtered using https://github.com/lucydot/mp-adventures/blob/main/imaginary_search.py and an initial analysis using https://github.com/lucydot/mp-adventures/blob/main/analysis_one.ipynb.

This imports the result of that search (`results.pickle`) and then collects further information about each compound (band gap, formula, lowest imaginary frequency, reciprocal space position of lowest imaginary frequency). Note that the tolerance for what counts as "imaginary" is pretty low - ie, some compounds will have imaginary modes resulting from numerical noise, but they can be filtered out at an additional step.

In [39]:
import pickle
import numpy as np
from mp_api import MPRester

tolerance=1e-5

In [40]:
with open("results.pickle", 'rb') as pickle_file:
    content = pickle.load(pickle_file)

In [42]:
results_dict = {}

with MPRester("os1XoXmCTeMm5rDO4kY9ClmfVKzuo5ek") as mpr:
    for MPID in content['MP-IDs_imag_freq__HSP']:
        
        f_list = []
        name_list = []
        info_dict = {}
        bs = mpr.phonon.get_data_by_id(MPID).ph_bs
        for branch in bs.branches:
            f_list.append(np.min(bs.bands[:,branch['start_index']]))
            name_list.append('start of '+branch["name"])
            f_list.append(np.min(bs.bands[:,branch['end_index']]))
            name_list.append('end of '+branch["name"])
        info_dict['lowest_freq_position'] = name_list[f_list.index(min(f_list))]
        info_dict['lowest_freq'] = min(f_list)
        
        try:
            summary = mpr.summary.search(material_ids=[MPID],fields=['formula_pretty','band_gap'])[0]
            info_dict['formula'] = summary.formula_pretty
            info_dict['band_gap'] = summary.band_gap
        except:
            pass
        
        results_dict[str(MPID)] = info_dict
        

Retrieving PhononBSDOSDoc documents: 100%|██████████| 1/1 [00:00<00:00, 16710.37it/s]


Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 12192.74it/s]


Retrieving PhononBSDOSDoc documents: 100%|██████████| 1/1 [00:00<00:00, 16448.25it/s]
Retrieving SummaryDoc documents: 0it [00:00, ?it/s]
Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 12052.60it/s]


Retrieving PhononBSDOSDoc documents: 100%|██████████| 1/1 [00:00<00:00, 12520.31it/s]


Retrieving PhononBSDOSDoc documents: 100%|██████████| 1/1 [00:00<00:00, 8924.05it/s]


Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 10407.70it/s]


Retrieving PhononBSDOSDoc documents: 100%|██████████| 1/1 [00:00<00:00, 6442.86it/s]


Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 445.40it/s]


Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 2272.10it/s]


In [43]:
with open('results_analysis.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle)